In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 73)

In [3]:
team_df = pd.read_pickle('data/team_stats_scraped.pickle')
game_df = pd.read_pickle('data/games_scraped.pickle')

In [4]:
game_df.shape

(29399, 26)

In [5]:
mask = (game_df.team == 'chi') & (game_df.year == 2019)
game_df[mask]

,team,year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,game_location,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st
4819,chi,2019,gnb,1,Thu,September 5,L,0-1,NaN,3.0,10.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,8.0,NaN,-20.0,9.0,-2.0
4820,chi,2019,den,2,Sun,September 15,W,1-1,@,16.0,14.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,4.0,NaN,4.0,-4.0,5.0
4821,chi,2019,was,3,Mon,September 23,W,2-1,@,31.0,15.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,8.0,NaN,9.0,11.0,-3.0
4822,chi,2019,min,4,Sun,September 29,W,3-1,NaN,16.0,6.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,4.0,NaN,-2.0,10.0,1.0
4823,chi,2019,rai,5,Sun,October 6,L,3-2,@,21.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0
4824,chi,2019,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4825,chi,2019,nor,7,Sun,October 20,L,3-3,NaN,25.0,36.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,4.0,NaN,-12.0,-13.0,10.0
4826,chi,2019,sdg,8,Sun,October 27,L,3-4,NaN,16.0,17.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,1.0,NaN,0.0,5.0,-6.0
4827,chi,2019,phi,9,Sun,November 3,L,3-5,@,14.0,22.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,1.0,NaN,-4.0,-10.0,2.0
4828,chi,2019,det,10,Sun,November 10,W,4-5,NaN,20.0,13.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,1.0,NaN,-0.0,6.0,-2.0


In [6]:
# clean the game_location column & apply change. def home_game(row):
def home_game(row):    
    if row == '@':
        return 0
    return 1

game_df['home'] = game_df.game_location.apply(home_game)
game_df.drop("game_location", axis=1, errors='ignore', inplace=True)

In [7]:
# add team_year identifier to each row (to easily groupby team/year)

team_years = game_df['team'] + '-' +  game_df['year'].astype(str) 
game_df.insert(loc=2, column='team_year', value=team_years)
game_df.sample()

,team,year,team_year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
27798,oti,1988,oti-1988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Datetime formatting strings:

- ``%B`` month fullname
- ``%-d`` day of month without leading zero\*
    - \*note: the ``-`` only works on Linux/OS-X...for windows replace with '#'
- ``%Y`` full year

In [8]:
# convert game date to datetime & add new column to DF


f = r"%B %d-%Y"
full_game_date = game_df.game_date + '-' + game_df.year.astype(str)
full_game_date = pd.to_datetime(full_game_date, format=f)
game_df.insert(loc=2, column='date', value=full_game_date)
game_df.sample(3)

,team,year,date,team_year,opp,week_num,game_day_of_week,game_date,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
10715,gnb,2016,2016-09-25,gnb-2016,det,3,Sun,September 25,W,2-1,34.0,27.0,17.0,324.0,201.0,123.0,NaN,22.0,418.0,368.0,50.0,1.0,1.0,NaN,17.0,-14.0,2.0,1
21288,phi,2001,2001-09-09,phi-2001,ram,1,Sun,September 9,L,0-1,17.0,20.0,19.0,334.0,277.0,57.0,3.0,22.0,364.0,282.0,82.0,2.0,4.0,OT,-7.0,-3.0,7.0,1
25000,sea,1990,1990-11-18,sea-1990,min,11,Sun,November 18,L,4-6,21.0,24.0,17.0,233.0,132.0,101.0,1.0,14.0,302.0,161.0,141.0,1.0,4.0,NaN,NaN,NaN,NaN,1


In [9]:
# now drop the game_date col
game_df.drop("game_date", axis=1, inplace=True, errors='ignore')
game_df.sample(2) # to confirm

,team,year,date,team_year,opp,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home
25325,sea,2008,2008-11-27,sea-2008,dal,13,Thu,L,2-10,9.0,34.0,19.0,322.0,242.0,80.0,2.0,24.0,447.0,331.0,116.0,1.0,4.0,NaN,-9.0,-15.0,2.0,0
25850,ram,1979,1979-09-23,ram-1979,tam,4,Sun,L,2-2,6.0,21.0,11.0,186.0,76.0,110.0,4.0,15.0,249.0,101.0,148.0,3.0,4.0,NaN,NaN,NaN,NaN,0


In [10]:
# add decade column
decades = game_df['year'] // 10
game_df.insert(loc=5, column='decade', value=decades)
game_df[['date', 'decade']].sample(10)

,date,decade
263,1978-11-05,197
14330,2004-01-02,200
3210,2011-10-09,201
1986,NaT,200
15324,2003-11-23,200
20807,1972-10-08,197
10642,2012-11-04,201
14889,1979-09-02,197
6976,1978-10-02,197
10353,1997-09-14,199


In [11]:
# add turnovers^2 col

game_df[['to2_off', 'to2_def']] = game_df[['to_off', 'to_def']].to_numpy() ** 2
game_df[['to_off', 'to2_off', 'to_def', 'to2_def']]

,to_off,to2_off,to_def,to2_def
0,5.0,25.0,6.0,36.0
1,5.0,25.0,6.0,36.0
2,6.0,36.0,3.0,9.0
3,6.0,36.0,NaN,NaN
4,3.0,9.0,2.0,4.0
...,...,...,...,...
29394,2.0,4.0,1.0,1.0
29395,4.0,16.0,2.0,4.0
29396,2.0,4.0,3.0,9.0
29397,NaN,NaN,NaN,NaN


In [12]:
# add unique game identifier that will be the same for both teams in the same game


def apply_game_id(row):
    teams = []
    teams.append(str(row['team']))
    teams.append(str(row['opp']))
    teams.sort()
    
    game_id = teams[0] + '-' +  teams[1] + '-' + str(row['date'])[:-9]
    return game_id

In [13]:
game_df['game_id'] = game_df.apply(apply_game_id, axis=1)
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,NaN,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,NaN,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,NaN,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,NaN,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1.0,-3.0,-4.0,1,4.0,1.0,sea-was-2020-12-20
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,-5.0,5.0,-7.0,1,16.0,4.0,car-was-2020-12-27
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,-8.0,11.0,3.0,0,4.0,9.0,phi-was-2020-01-03
29397,was,2020,NaT,was-2020,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,nan-was-


In [14]:
cat_game_outcome = game_df.loc[:, ['game_outcome']]
cat_game_outcome.value_counts()

game_outcome
L               13773
W               13773
T                 254
dtype: int64

In [15]:
# convert game_outcome col to binary to avoid multicolinearity

game_df[['result_tie', 'result_win']] = pd.get_dummies(game_df.game_outcome, drop_first=True)
game_df.sample(10)

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win
5515,cin,2008,NaT,cin-2008,NaN,200,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,cin-nan-,0,0
24317,sfo,1996,1996-09-29,sfo-1996,atl,199,5,Sun,W,3-1,39.0,17.0,16.0,396.0,246.0,150.0,NaN,18.0,317.0,191.0,126.0,4.0,4.0,NaN,6.0,6.0,8.0,1,NaN,16.0,atl-sfo-1996-09-29,0,1
4832,chi,2019,2019-12-05,chi-2019,dal,201,14,Thu,W,7-6,31.0,24.0,24.0,382.0,231.0,151.0,2.0,22.0,408.0,326.0,82.0,NaN,8.0,NaN,10.0,-10.0,8.0,1,4.0,NaN,chi-dal-2019-12-05,0,1
11219,clt,1965,1965-10-10,clt-1965,det,196,4,Sun,W,3-1,31.0,7.0,17.0,363.0,294.0,69.0,2.0,14.0,155.0,89.0,66.0,4.0,NaN,NaN,NaN,NaN,NaN,1,4.0,16.0,clt-det-1965-10-10,0,1
11090,htx,2018,2018-09-09,htx-2018,nwe,201,1,Sun,L,0-1,20.0,27.0,21.0,325.0,158.0,167.0,2.0,23.0,389.0,267.0,122.0,3.0,1.0,NaN,-8.0,-4.0,6.0,0,4.0,9.0,htx-nwe-2018-09-09,0,0
10886,htx,2006,2006-11-26,htx-2006,nyj,200,12,Sun,L,3-8,11.0,26.0,21.0,334.0,309.0,25.0,1.0,17.0,304.0,277.0,27.0,NaN,1.0,NaN,-4.0,-8.0,3.0,0,1.0,NaN,htx-nyj-2006-11-26,0,0
8459,den,2003,2003-11-30,den-2003,rai,200,13,Sun,W,7-5,22.0,8.0,20.0,287.0,94.0,193.0,1.0,12.0,262.0,142.0,120.0,3.0,4.0,NaN,-0.0,19.0,-7.0,0,1.0,9.0,den-rai-2003-11-30,0,1
3423,car,1997,1997-09-21,car-1997,kan,199,4,Sun,L,2-2,14.0,35.0,24.0,364.0,278.0,86.0,5.0,18.0,331.0,214.0,117.0,1.0,1.0,NaN,-19.0,-0.0,-2.0,1,25.0,1.0,car-kan-1997-09-21,0,0
14282,mia,2002,2002-09-22,mia-2002,nyj,200,3,Sun,W,3-0,30.0,3.0,21.0,394.0,183.0,211.0,NaN,13.0,189.0,131.0,58.0,4.0,1.0,NaN,12.0,20.0,-5.0,1,NaN,16.0,mia-nyj-2002-09-22,0,1
21210,phi,1996,1996-11-24,phi-1996,crd,199,13,Sun,L,7-5,30.0,36.0,22.0,359.0,292.0,67.0,2.0,27.0,447.0,340.0,107.0,1.0,4.0,NaN,0.0,-12.0,6.0,0,4.0,1.0,crd-phi-1996-11-24,0,0


In [16]:
mask = game_df.game_outcome == "T"
game_df[mask].head()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win
9,crd,1960,1960-11-27,crd-1960,cle,196,10,Sun,T,5-4-1,17.0,17.0,21.0,322.0,127.0,195.0,6.0,17.0,274.0,88.0,186.0,2.0,NaN,NaN,NaN,NaN,NaN,1,36.0,4.0,cle-crd-1960-11-27,1,0
30,crd,1962,1962-10-14,crd-1962,was,196,5,Sun,T,1-3-1,17.0,17.0,16.0,309.0,251.0,58.0,2.0,15.0,321.0,259.0,62.0,1.0,NaN,NaN,NaN,NaN,NaN,1,4.0,1.0,crd-was-1962-10-14,1,0
55,crd,1964,1964-09-20,crd-1964,cle,196,2,Sun,T,1-0-1,33.0,33.0,17.0,344.0,230.0,114.0,3.0,18.0,281.0,166.0,115.0,3.0,NaN,NaN,NaN,NaN,NaN,0,9.0,9.0,cle-crd-1964-09-20,1,0
63,crd,1964,1964-11-15,crd-1964,nyg,196,10,Sun,T,5-3-2,10.0,10.0,12.0,143.0,105.0,38.0,3.0,12.0,212.0,131.0,81.0,3.0,NaN,NaN,NaN,NaN,NaN,1,9.0,9.0,crd-nyg-1964-11-15,1,0
87,crd,1966,1966-10-16,crd-1966,dal,196,6,Sun,T,5-0-1,10.0,10.0,13.0,175.0,130.0,45.0,NaN,17.0,326.0,191.0,135.0,4.0,NaN,NaN,NaN,NaN,NaN,1,NaN,16.0,crd-dal-1966-10-16,1,0


In [17]:


game_df = (game_df.assign(wins=game_df.team_record.str.split('-').str.get(0),
                          losses=game_df.team_record.str.split('-').str.get(1),
                          ties=game_df.team_record.str.split('-').str.get(2)))
game_df.sample()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties
9089,det,1981,1981-12-12,det-1981,min,198,15,Sat,W,8-7,45.0,7.0,29.0,476.0,271.0,205.0,4.0,16.0,216.0,144.0,72.0,4.0,4.0,NaN,NaN,NaN,NaN,1,16.0,16.0,det-min-1981-12-12,0,1,8,7,NaN


Okay, almost there. Our ties are coming through, but we get NaNs for teams that haven't tied, so we'll need to replace those with zeros.

In [18]:
# fill ties NaNs with zero
game_df['ties'] = game_df.ties.fillna(0)

In [19]:
# add 'margin' col
margins = game_df.pts_off - game_df.pts_def
game_df.insert(loc=11, column='margin', value=margins)
game_df[mask].head()

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties
9,crd,1960,1960-11-27,crd-1960,cle,196,10,Sun,T,5-4-1,17.0,0.0,17.0,21.0,322.0,127.0,195.0,6.0,17.0,274.0,88.0,186.0,2.0,NaN,NaN,NaN,NaN,NaN,1,36.0,4.0,cle-crd-1960-11-27,1,0,5,4,1
30,crd,1962,1962-10-14,crd-1962,was,196,5,Sun,T,1-3-1,17.0,0.0,17.0,16.0,309.0,251.0,58.0,2.0,15.0,321.0,259.0,62.0,1.0,NaN,NaN,NaN,NaN,NaN,1,4.0,1.0,crd-was-1962-10-14,1,0,1,3,1
55,crd,1964,1964-09-20,crd-1964,cle,196,2,Sun,T,1-0-1,33.0,0.0,33.0,17.0,344.0,230.0,114.0,3.0,18.0,281.0,166.0,115.0,3.0,NaN,NaN,NaN,NaN,NaN,0,9.0,9.0,cle-crd-1964-09-20,1,0,1,0,1
63,crd,1964,1964-11-15,crd-1964,nyg,196,10,Sun,T,5-3-2,10.0,0.0,10.0,12.0,143.0,105.0,38.0,3.0,12.0,212.0,131.0,81.0,3.0,NaN,NaN,NaN,NaN,NaN,1,9.0,9.0,crd-nyg-1964-11-15,1,0,5,3,2
87,crd,1966,1966-10-16,crd-1966,dal,196,6,Sun,T,5-0-1,10.0,0.0,10.0,13.0,175.0,130.0,45.0,NaN,17.0,326.0,191.0,135.0,4.0,NaN,NaN,NaN,NaN,NaN,1,NaN,16.0,crd-dal-1966-10-16,1,0,5,0,1


In [20]:
# drop rows for bye weeks
game_df.dropna(axis=0, how='any', subset=['game_outcome'], inplace=True)


# drop exp_pts cols
game_df.drop(['exp_pts_off', 'exp_pts_def', 'exp_pts_st'], axis=1, inplace=True)

In [21]:
# get integer week_num col

def apply_season_week_col(row):
    week_val = row['week_num']
    if week_val.isdigit():
        return int(week_val)
    else:
        return 0
    
game_df['season_week'] = game_df.apply(apply_season_week_col, axis=1)
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,22.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23,0,1,1,0,0,1
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,-21.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02,0,0,1,1,0,2
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,-4.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09,0,0,1,2,0,3
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,-13.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16,0,0,1,3,0,4
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,2.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23,0,1,2,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29393,was,2020,2020-12-13,was-2020,sfo,202,14,Sun,W,6-7,23.0,8.0,15.0,12.0,193.0,95.0,98.0,1.0,21.0,344.0,236.0,108.0,3.0,4.0,NaN,0,1.0,9.0,sfo-was-2020-12-13,0,1,6,7,0,14
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,-5.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1,4.0,1.0,sea-was-2020-12-20,0,0,6,8,0,15
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,-7.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,1,16.0,4.0,car-was-2020-12-27,0,0,6,9,0,16
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,6.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,0,4.0,9.0,phi-was-2020-01-03,0,1,7,9,0,17


In [22]:
# add 'playoff_game' column
game_df['playoff_game'] = game_df['season_week'] == 0
game_df

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week,playoff_game
0,crd,1960,1960-09-23,crd-1960,ram,196,1,Fri,W,1-0,43.0,22.0,21.0,20.0,430.0,281.0,149.0,5.0,16.0,238.0,188.0,50.0,6.0,NaN,NaN,0,25.0,36.0,crd-ram-1960-09-23,0,1,1,0,0,1,False
1,crd,1960,1960-10-02,crd-1960,nyg,196,2,Sun,L,1-1,14.0,-21.0,35.0,11.0,222.0,115.0,107.0,5.0,18.0,405.0,268.0,137.0,6.0,NaN,NaN,1,25.0,36.0,crd-nyg-1960-10-02,0,0,1,1,0,2,False
2,crd,1960,1960-10-09,crd-1960,phi,196,3,Sun,L,1-2,27.0,-4.0,31.0,20.0,364.0,145.0,219.0,6.0,14.0,293.0,199.0,94.0,3.0,NaN,NaN,0,36.0,9.0,crd-phi-1960-10-09,0,0,1,2,0,3,False
3,crd,1960,1960-10-16,crd-1960,pit,196,4,Sun,L,1-3,14.0,-13.0,27.0,20.0,295.0,154.0,141.0,6.0,12.0,237.0,146.0,91.0,NaN,NaN,NaN,0,36.0,NaN,crd-pit-1960-10-16,0,0,1,3,0,4,False
4,crd,1960,1960-10-23,crd-1960,dal,196,5,Sun,W,2-3,12.0,2.0,10.0,17.0,306.0,80.0,226.0,3.0,9.0,175.0,124.0,51.0,2.0,NaN,NaN,1,9.0,4.0,crd-dal-1960-10-23,0,1,2,3,0,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29393,was,2020,2020-12-13,was-2020,sfo,202,14,Sun,W,6-7,23.0,8.0,15.0,12.0,193.0,95.0,98.0,1.0,21.0,344.0,236.0,108.0,3.0,4.0,NaN,0,1.0,9.0,sfo-was-2020-12-13,0,1,6,7,0,14,False
29394,was,2020,2020-12-20,was-2020,sea,202,15,Sun,L,6-8,15.0,-5.0,20.0,26.0,353.0,269.0,84.0,2.0,16.0,302.0,121.0,181.0,1.0,1.0,NaN,1,4.0,1.0,sea-was-2020-12-20,0,0,6,8,0,15,False
29395,was,2020,2020-12-27,was-2020,car,202,16,Sun,L,6-9,13.0,-7.0,20.0,20.0,386.0,278.0,108.0,4.0,19.0,280.0,167.0,113.0,2.0,4.0,NaN,1,16.0,4.0,car-was-2020-12-27,0,0,6,9,0,16,False
29396,was,2020,2020-01-03,was-2020,phi,202,17,Sun,W,7-9,20.0,6.0,14.0,16.0,248.0,145.0,103.0,2.0,16.0,216.0,98.0,118.0,3.0,8.0,NaN,0,4.0,9.0,phi-was-2020-01-03,0,1,7,9,0,17,False


In [23]:
# add 'prev_week' cols

cols_to_shift = ['wins', 'losses', 'season_week', 'ties', 'pts_off', 'pts_def',
       'margin', 'first_down_off', 'yards_off', 'pass_yds_off', 'rush_yds_off',
       'to_off', 'first_down_def', 'yards_def', 'pass_yds_def', 'rush_yds_def',
       'to_def', 'result_tie', 'result_win']

for col in cols_to_shift:
    new_col = 'prev_' + col
    game_df[new_col] = game_df.groupby('team_year')[col].apply(lambda grp: grp.shift(1))

In [24]:
# add 'off_bye' (boolean) col
    
def apply_off_bye(row):
    if row['season_week'] > 1: # playoff games have week_num = 0
        off_bye = row['season_week'] - row['prev_season_week'] == 2
    else:
        off_bye = False
    if off_bye == True:
        return 1
    return 0


game_df['off_bye'] = game_df[['season_week', 'prev_season_week']].apply(apply_off_bye, axis=1)



In [25]:
mask = game_df.team_year == 'chi-2019'
game_df[mask]

,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week,playoff_game,prev_wins,prev_losses,prev_season_week,prev_ties,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,off_bye
4819,chi,2019,2019-09-05,chi-2019,gnb,201,1,Thu,L,0-1,3.0,-7.0,10.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,8.0,NaN,1,1.0,NaN,chi-gnb-2019-09-05,0,0,0,1,0,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4820,chi,2019,2019-09-15,chi-2019,den,201,2,Sun,W,1-1,16.0,2.0,14.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,4.0,NaN,0,NaN,1.0,chi-den-2019-09-15,0,1,1,1,0,2,False,0,1,1.0,0,3.0,10.0,-7.0,16.0,254.0,208.0,46.0,1.0,13.0,213.0,166.0,47.0,NaN,0.0,0.0,0
4821,chi,2019,2019-09-23,chi-2019,was,201,3,Mon,W,2-1,31.0,16.0,15.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,8.0,NaN,0,1.0,25.0,chi-was-2019-09-23,0,1,2,1,0,3,False,1,1,2.0,0,16.0,14.0,2.0,18.0,273.0,120.0,153.0,NaN,27.0,372.0,282.0,90.0,1.0,0.0,1.0,0
4822,chi,2019,2019-09-29,chi-2019,min,201,4,Sun,W,3-1,16.0,10.0,6.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,4.0,NaN,1,NaN,4.0,chi-min-2019-09-29,0,1,3,1,0,4,False,2,1,3.0,0,31.0,15.0,16.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,0.0,1.0,0
4823,chi,2019,2019-10-06,chi-2019,rai,201,5,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,0,4.0,4.0,chi-rai-2019-10-06,0,0,3,2,0,5,False,3,1,4.0,0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,0.0,1.0,0
4825,chi,2019,2019-10-20,chi-2019,nor,201,7,Sun,L,3-3,25.0,-11.0,36.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,4.0,NaN,1,4.0,NaN,chi-nor-2019-10-20,0,0,3,3,0,7,False,3,2,5.0,0,21.0,24.0,-3.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,0.0,0.0,1
4826,chi,2019,2019-10-27,chi-2019,sdg,201,8,Sun,L,3-4,16.0,-1.0,17.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,1.0,NaN,1,4.0,1.0,chi-sdg-2019-10-27,0,0,3,4,0,8,False,3,3,7.0,0,25.0,36.0,-11.0,16.0,252.0,235.0,17.0,2.0,24.0,424.0,273.0,151.0,NaN,0.0,0.0,0
4827,chi,2019,2019-11-03,chi-2019,phi,201,9,Sun,L,3-5,14.0,-8.0,22.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,1.0,NaN,0,1.0,NaN,chi-phi-2019-11-03,0,0,3,5,0,9,False,3,4,8.0,0,16.0,17.0,-1.0,26.0,388.0,226.0,162.0,2.0,11.0,231.0,195.0,36.0,1.0,0.0,0.0,0
4828,chi,2019,2019-11-10,chi-2019,det,201,10,Sun,W,4-5,20.0,7.0,13.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,1.0,NaN,1,NaN,1.0,chi-det-2019-11-10,0,1,4,5,0,10,False,3,5,9.0,0,14.0,22.0,-8.0,10.0,164.0,102.0,62.0,1.0,26.0,373.0,227.0,146.0,NaN,0.0,0.0,0
4829,chi,2019,2019-11-17,chi-2019,ram,201,11,Sun,L,4-6,7.0,-10.0,17.0,17.0,267.0,193.0,74.0,1.0,13.0,283.0,173.0,110.0,2.0,8.0,NaN,0,1.0,4.0,chi-ram-2019-11-17,0,0,4,6,0,11,False,4,5,10.0,0,20.0,13.0,7.0,13.0,226.0,145.0,81.0,NaN,21.0,357.0,259.0,98.0,1.0,0.0,1.0,0


In [26]:
game_df.columns

Index(['team', 'year', 'date', 'team_year', 'opp', 'decade', 'week_num',
       'game_day_of_week', 'game_outcome', 'team_record', 'pts_off', 'margin',
       'pts_def', 'first_down_off', 'yards_off', 'pass_yds_off',
       'rush_yds_off', 'to_off', 'first_down_def', 'yards_def', 'pass_yds_def',
       'rush_yds_def', 'to_def', 'game_time', 'overtime', 'home', 'to2_off',
       'to2_def', 'game_id', 'result_tie', 'result_win', 'wins', 'losses',
       'ties', 'season_week', 'playoff_game', 'prev_wins', 'prev_losses',
       'prev_season_week', 'prev_ties', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'off_bye'],
      dtype='object')

In [27]:
# create rolling three feature for these stats:
# using margin instead of wins here... if we want to get num wins in last three weeks,
#  will need to do differently using sum instead of mean

# don't use prev_ columns, we will just shift them down when we use .transform()
roll_cols = ['result_win', 'result_tie', 'pts_off', 'pts_def', 'margin', 'first_down_off', 'yards_off', 
             'pass_yds_off', 'rush_yds_off', 'to_off', 'to2_off', 'yards_def', 'pass_yds_def',
             'rush_yds_def', 'to_def', 'to2_def']

In [28]:
roll3_cols = ['roll3_' + col_name for col_name in roll_cols]

game_df[roll3_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).rolling(3).mean(), 3)))


In [29]:
# get rolling wins (sum) (commented out to try using mean)

rolling_wins = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_wins', value=rolling_wins)

rolling_ties = (game_df.groupby('team_year')['result_win']
                      .transform(lambda x: round(x.shift(1).rolling(3).sum(), 3)))

game_df.insert(loc=53, column='roll3_num_ties', value=rolling_ties)

Now let's add more columns with **Expontentially Weighted Moving Averages (EWMA)** of our stats. This is similar to taking a rolling mean of a team's previous 6 performances, but EWMA weights the most recent game in the window much higher than the week before that, and so on.

For this, we will use a 3-16 week span (greedy). My hypothesis is that the most recent performances are the best predictors, and this should help further quantify recent performance.

In [30]:
# add ewma cols

ewma_cols = ['ewma_' + col_name for col_name in roll_cols]

game_df[ewma_cols] = (game_df.groupby('team_year')[roll_cols]
                      .transform(lambda x: round(x.shift(1).ewm(span=16, min_periods=3).mean(), 3)))


In [31]:
mask = game_df.team_year == 'nor-2014'
game_df[mask][['week_num', 'team', 'pass_yds_off', 'prev_pass_yds_off', 'ewma_pass_yds_off', 'roll3_pass_yds_off']].head(18)

,week_num,team,pass_yds_off,prev_pass_yds_off,ewma_pass_yds_off,roll3_pass_yds_off
17457,1,nor,333.0,NaN,NaN,NaN
17458,2,nor,223.0,333.0,NaN,NaN
17459,3,nor,288.0,223.0,NaN,NaN
17460,4,nor,334.0,288.0,279.612,281.333
17461,5,nor,371.0,334.0,295.858,281.667
17463,7,nor,335.0,371.0,314.862,331.000
17464,8,nor,302.0,335.0,319.348,346.667
17465,9,nor,270.0,302.0,315.851,336.000
17466,10,nor,287.0,270.0,307.324,302.333
17467,11,nor,255.0,287.0,303.786,286.333


In [32]:
np.mean([197, 120, 208])

175.0

Now, let's delete the first three rows of each team-year. We have to do this because we will be using at least some rolling columns for our model.

In [33]:
game_df.dropna(axis=0, how="any", subset=["roll3_pts_off"], inplace=True)

Looks good!!

**Now, we have to solve a pretty complex problem.**

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

Let's take a look at an example.

In [34]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask]


,team,year,date,team_year,opp,decade,week_num,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,home,to2_off,to2_def,game_id,result_tie,result_win,wins,losses,ties,season_week,playoff_game,prev_wins,prev_losses,prev_season_week,prev_ties,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,roll3_num_ties,roll3_num_wins,prev_result_tie,prev_result_win,off_bye,roll3_result_win,roll3_result_tie,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_to2_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,roll3_to2_def,ewma_result_win,ewma_result_tie,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_to2_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def,ewma_to2_def
4823,chi,2019,2019-10-06,chi-2019,rai,201,5,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,0,4.0,4.0,chi-rai-2019-10-06,0,0,3,2,0,5,False,3,1,4.0,0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,NaN,15.0,222.0,182.0,40.0,2.0,3.0,3.0,0.0,1.0,0,1.000,0.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,NaN,NaN,316.667,250.333,66.333,2.667,10.0,0.795,0.0,17.286,11.053,6.233,18.082,274.496,184.250,90.246,NaN,NaN,290.353,229.646,60.707,2.702,10.086
20606,rai,2019,2019-10-06,rai-2019,chi,201,5,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,1,4.0,4.0,chi-rai-2019-10-06,0,1,3,2,0,5,False,2,2,4.0,0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,1.0,1.0,0.0,1.0,0,0.333,0.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,2.0,399.333,291.667,107.667,NaN,NaN,0.504,0.0,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.293,1.878,384.007,277.499,106.508,NaN,NaN


So, in the above table, we need to get all of the roll3_x and ewma_x columns from the adjacent tables, respectively. We'll prefix these with opp_.

In [35]:
game_df.columns

Index(['team', 'year', 'date', 'team_year', 'opp', 'decade', 'week_num',
       'game_day_of_week', 'game_outcome', 'team_record', 'pts_off', 'margin',
       'pts_def', 'first_down_off', 'yards_off', 'pass_yds_off',
       'rush_yds_off', 'to_off', 'first_down_def', 'yards_def', 'pass_yds_def',
       'rush_yds_def', 'to_def', 'game_time', 'overtime', 'home', 'to2_off',
       'to2_def', 'game_id', 'result_tie', 'result_win', 'wins', 'losses',
       'ties', 'season_week', 'playoff_game', 'prev_wins', 'prev_losses',
       'prev_season_week', 'prev_ties', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'roll3_num_ties', 'roll3_num_wins',
       'prev_result_tie', 'prev_result_win', 'off_bye', 'roll3_result_win',
       'roll3_result_tie', 'roll3_pts_off', 'roll3_pts

In [36]:
opp_pull_cols = ['game_id', 'team', 'opp', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_result_win', 'roll3_num_wins', 'roll3_result_tie', 
       'roll3_num_ties', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',
       'roll3_rush_yds_off', 'roll3_to_off', 'roll3_yards_def',
       'roll3_pass_yds_def', 'roll3_rush_yds_def', 'roll3_to_def', 'ewma_result_win',
       'ewma_result_tie', 'ewma_num_wins', 'ewma_num_ties', 
       'ewma_pts_off', 'ewma_pts_def', 'ewma_margin', 'ewma_first_down_off',
       'ewma_yards_off', 'ewma_pass_yds_off', 'ewma_rush_yds_off',
       'ewma_to_off', 'ewma_yards_def', 'ewma_pass_yds_def',
       'ewma_rush_yds_def', 'ewma_to_def']

In [37]:
# # convert all cols to float

# game_df[opp_pull_cols] = game_df[opp_pull_cols].fillna(0)
# for col in opp_pull_cols[3:]:
#     game_df[col] = game_df[col].astype(float)

KeyError: "['ewma_num_ties', 'ewma_num_wins'] not in index"

**Let's solve this using `pd.merge`, using a self-join.**

In [38]:
game_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

KeyError: "['ewma_num_ties', 'ewma_num_wins'] not in index"

Let's make sure it worked by checking a couple of random game_ids.

In [39]:
mask = (game_df.game_id == 'dal-nyg-2019-11-04')

game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

KeyError: "['roll3_margin_opp', 'prev_rush_yds_off_opp'] not in index"

In [40]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

KeyError: "['roll3_margin_opp', 'prev_rush_yds_off_opp'] not in index"

In [ ]:
mask = (game_df.game_id == 'chi-gnb-1960-12-04')
game_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

## Now let's take another look to make sure everything looks good.

If it does, we're ready to start modeling!

In [ ]:
mask = game_df['team_year'] == 'chi-2019'
game_df[mask]

Looks great. Now it's time to start modeling our data! Head over to `model_exploration.ipynb` to continue following along.

In [ ]:
game_df

In [ ]:
game_df.drop_duplicates(subset=['game_id'], inplace=True)

In [ ]:
game_df.shape